In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Load the csv exported in Part I to a DataFrame
vacation_df = pd.DataFrame(pd.read_csv('output_data/cities.csv'))
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,peleduy,59.6289,112.7406,34.03,100,100,1.19,RU,1632783564
1,1,atasu,48.6814,71.6433,49.64,60,70,8.75,KZ,1632783564
2,2,lebu,-37.6167,-73.6500,50.13,76,10,18.95,CL,1632783564
3,3,busselton,-33.6500,115.3333,51.08,82,75,8.77,AU,1632783564
4,4,arraial do cabo,-22.9661,-42.0278,73.36,88,0,14.97,BR,1632783564


Humidity Heatmap
-Configure gmaps.
-Use the Lat and Lng as locations and Humidity as the weight.
-Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria
-Narrow down the cities to fit weather conditions.
-Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = vacation_df[(vacation_df["Max Temp"] < 80) & (vacation_df["Max Temp"] > 70) \
                                    & (vacation_df["Wind Speed"] < 10) \
                                    & (vacation_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,tiznit,29.5833,-9.5000,77.32,25,0,1.03,MA,1632783568
362,362,paralimni,35.0375,33.9833,71.11,87,0,3.00,CY,1632783594
423,423,erzin,36.9559,36.2000,73.00,72,0,1.59,TR,1632783600
424,424,pinehurst,35.1954,-79.4695,79.99,66,0,3.94,US,1632783600
430,430,asyut,27.1810,31.1837,73.58,38,0,7.94,EG,1632783600
449,449,san vicente,-26.6167,-54.1333,72.75,44,0,1.23,AR,1632783609
473,473,ardakan,32.3100,54.0175,72.55,29,0,4.25,IR,1632783611
483,483,santa fe,-31.6333,-60.7000,79.00,50,0,3.00,AR,1632783356
490,490,jalu,29.0331,21.5482,72.23,51,0,9.51,LY,1632783613


Hotel Map
-Store into variable named hotel_df.
-Add a "Hotel Name" column to the DataFrame.
-Set parameters to search for hotels with 5000 meters.
-Hit the Google Places API for each city's coordinates.
-Store the first Hotel result into the DataFrame.
-Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
52,tiznit,MA,29.5833,-9.5000,
362,paralimni,CY,35.0375,33.9833,
423,erzin,TR,36.9559,36.2000,
424,pinehurst,US,35.1954,-79.4695,
430,asyut,EG,27.1810,31.1837,
449,san vicente,AR,-26.6167,-54.1333,
473,ardakan,IR,32.3100,54.0175,
483,santa fe,AR,-31.6333,-60.7000,
490,jalu,LY,29.0331,21.5482,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
52,tiznit,MA,29.5833,-9.5000,
362,paralimni,CY,35.0375,33.9833,Amore Hotel Apartments
423,erzin,TR,36.9559,36.2000,Artemis Otel
424,pinehurst,US,35.1954,-79.4695,SpringHill Suites by Marriott Pinehurst Southe...
430,asyut,EG,27.1810,31.1837,Florence Hotel Assiut
449,san vicente,AR,-26.6167,-54.1333,Complejo Turístico Farelluk
473,ardakan,IR,32.3100,54.0175,Ardakan Hotel
483,santa fe,AR,-31.6333,-60.7000,Conquistador Hotel
490,jalu,LY,29.0331,21.5482,صاحبة الفخامة


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))